In [1]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../../')
%run setup.py install
os.chdir(owd)

In [2]:
%matplotlib inline
import numpy as np
import scipy
from matplotlib import pyplot as plt
import pyross
import time 

In [3]:
M  = 2                  # the population has two age groups
N  =  5e4           # and this is the total population

# correct params

beta  = 0.02         # infection rate
gIa   = 1./7            # recovery rate of asymptomatic infectives
gIs   = 1./7            # recovery rate of asymptomatic infectives
alpha = 0.2          # fraction of asymptomatic infectives
fsa   = 0.8          # the self-isolation parameter

# set the age structure
fi = np.array([0.25, 0.75])  # fraction of population in age age group
Ni = N*fi

# set the contact structure
C = np.array([[18., 9.], [3., 12.]])

Tf = 100
Nf = Tf+1
steps = 101 

# set up initial condition
Ia0 = np.array([10, 10])  # each age group has asymptomatic infectives
Is0 = np.array([10, 10])   # and also symptomatic infectives
R0  = np.array([0, 0])  # there are no recovered individuals initially
S0  = Ni - (Ia0 + Is0 + R0)

def contactMatrix(t):
    return C

parameters = {'alpha':alpha, 'beta':beta, 'gIa':gIa, 'gIs':gIs,'fsa':fsa}

# use pyross stochastic to generate traj and save 
sto_model = pyross.stochastic.SIR(parameters, M, Ni)
data = sto_model.simulate(S0, Ia0, Is0, contactMatrix, Tf, Nf)
data_array = data['X']
np.save('sto_traj.npy', data_array)
parameters = {'alpha':alpha, 'beta':beta, 'gIa':gIa, 'gIs':gIs,'fsa':fsa}

In [4]:
# load the data and rescale to intensive variables 
x = np.load('sto_traj.npy').astype('float')
x = x/N

# initialise the estimator 
estimator = pyross.inference.SIR(parameters, M, fi, int(N), steps)

In [6]:
# take a guess 
beta_g = 0.1
gIa_g = 0.1
gIs_g = 0.1
alpha_g = 0.4
guess = [alpha_g, beta_g, gIa_g, gIs_g]

# inference 
MAPs, nit = estimator.inference(guess, x, Tf, Nf, contactMatrix, ftol=1e-6) # currently only guess four parameters
print(MAPs) # best guess 
print(nit) # number of iterations of the optimization run 

[0.1942352  0.02005099 0.13917373 0.14391947]
28


# Gaussian approximation for the $M$-variate Ornstein-Uhlenbeck process

$\ln P(X|M_i) \approx \ln P(X|\theta,M_i)|_{\theta^*} -\frac{1}{2}\ln\det \boldsymbol{A} + \frac{M}{2}\ln 2\pi$

With $\ln P(X|\theta,M_i)|_{\theta^*}$ the best fit likelihood, in our case of flat parameter priors, this is simply the posterior evaluated at the MAP estimates. The Hessian matrix $\boldsymbol{A}=-\nabla\nabla\ln P(\theta|X,M_i)|_{\theta^*}$ is calculated using finite differences.

In [7]:
start = time.time()
hess = estimator.hessian(MAPs, x, Tf, Nf, contactMatrix, eps=1.e-3)
print(hess)
end = time.time()

print('time: ', end-start)

[[ 1.36732301e+05  3.83667184e+03 -1.31446506e+05  1.26845653e+05]
 [ 3.83667184e+03  8.49475895e+07 -3.64532658e+03 -2.81708134e+04]
 [-1.31446506e+05 -3.64532658e+03  2.48019115e+05  1.41765579e+05]
 [ 1.26845653e+05 -2.81708134e+04  1.41765579e+05  1.38210465e+06]]
time:  14.228440999984741


In [8]:
evidence = estimator.log_G_evidence(MAPs, x, Tf, Nf, contactMatrix, eps=1.e-3)
print(evidence)

1861.6837536155901


# Nested sampling 

Compute the logarithm of the evidence $P(X|M_i)=\int P(X|\theta,M_i)P(\theta|M_i)d\theta$ exactly using nestle, an excellent implementation of the nested sampling algorithm. 

With an appropriate prior transform function ($x\in[0,1]$) we supply a flat prior with the same interval for all parameters. This is of course easily changed to various priors. 

In [ ]:
# takes 30 mins 
# not tested yet 
evidence = estimator.log_NS_evidence(self, x, Tf, Nf, contactMatrix, UB=1., LB=0.001, P=4)
print(evidence)